In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics

In [2]:
qb_college = pd.read_csv('qb_college.csv')
rookie_season = pd.read_csv('Rookie_QB_Dataset.csv')

In [3]:
qb_college.head()

,QB,Year,Round,Years Played,Cmp,Att,Pct,Yds,Y/A,AY/A,TD,Int,Rate
0,Malik Willis,2022,3,4,388,618,62.8,5176,8.4,8.6,48,18,152.9
1,Kenny Pickett,2022,1,5,1045,1674,62.4,12303,7.3,7.5,81,32,136.3
2,Sam Howell,2022,5,3,713,1117,63.8,10283,9.2,9.9,92,23,164.2
3,Desmond Ridder,2022,3,4,810,1304,62.1,10239,7.9,8.2,87,28,145.8
4,Matt Corral,2022,3,4,614,912,67.3,8287,9.1,9.2,57,23,159.2


In [4]:
rookie_season.head()

,NAME,SEASON,TM,GP,CMP,ATT,PCT,YDS,AVG,YDS/G,TD,TD%,INT,INT%,SCK,SCKY,RATE
0,Bryce Perkins,2022,LAR,5,19,34,55.9,161,4.7,32.2,1,2.9,2,5.9,6,50,53.7
1,Kyle Trask,2022,TB,1,3,9,33.3,23,2.6,23.0,0,0.0,0,0.0,0,0,42.4
2,Sam Howell,2022,WAS,1,11,19,57.9,169,8.9,169.0,1,5.3,1,5.3,3,11,83.0
3,Skylar Thompson,2022,MIA,7,60,105,57.1,534,5.1,76.3,1,1.0,3,2.9,6,40,62.2
4,Bailey Zappe,2022,NE,4,65,92,70.7,781,8.5,195.3,5,5.4,3,3.3,6,40,100.9


In [5]:
avg_rookie_rate = np.mean(rookie_season["RATE"])
avg_rookie_rate

70.32047244094488

In [6]:
above_avg = []
for i in range(len(rookie_season)):
    if (rookie_season["RATE"][i] > avg_rookie_rate and rookie_season["ATT"][i] > 100):
        above_avg.append(1)
    else:
        above_avg.append(0)
        
rookie_season['SUCCESS'] = above_avg
rookie_season.head()

,NAME,SEASON,TM,GP,CMP,ATT,PCT,YDS,AVG,YDS/G,TD,TD%,INT,INT%,SCK,SCKY,RATE,SUCCESS
0,Bryce Perkins,2022,LAR,5,19,34,55.9,161,4.7,32.2,1,2.9,2,5.9,6,50,53.7,0
1,Kyle Trask,2022,TB,1,3,9,33.3,23,2.6,23.0,0,0.0,0,0.0,0,0,42.4,0
2,Sam Howell,2022,WAS,1,11,19,57.9,169,8.9,169.0,1,5.3,1,5.3,3,11,83.0,0
3,Skylar Thompson,2022,MIA,7,60,105,57.1,534,5.1,76.3,1,1.0,3,2.9,6,40,62.2,0
4,Bailey Zappe,2022,NE,4,65,92,70.7,781,8.5,195.3,5,5.4,3,3.3,6,40,100.9,0


In [7]:
add_success = []

for i in range(len(qb_college)):
    got_value = 0
    for j in range(len(rookie_season)):
        if (qb_college['QB'][i] == rookie_season["NAME"][j]):
            add_success.append(rookie_season["SUCCESS"][j])
            got_value += 1
            break
    if (got_value < 1):
        add_success.append(0)

qb_college["Success"] = add_success
qb_college.head()

,QB,Year,Round,Years Played,Cmp,Att,Pct,Yds,Y/A,AY/A,TD,Int,Rate,Success
0,Malik Willis,2022,3,4,388,618,62.8,5176,8.4,8.6,48,18,152.9,0
1,Kenny Pickett,2022,1,5,1045,1674,62.4,12303,7.3,7.5,81,32,136.3,1
2,Sam Howell,2022,5,3,713,1117,63.8,10283,9.2,9.9,92,23,164.2,0
3,Desmond Ridder,2022,3,4,810,1304,62.1,10239,7.9,8.2,87,28,145.8,1
4,Matt Corral,2022,3,4,614,912,67.3,8287,9.1,9.2,57,23,159.2,0


# Models

In [8]:
qbs = qb_college['QB']
successful = qb_college['Success']
college_stats = qb_college.drop(columns=['QB', 'Success'])

In [9]:
train_vectors, test_vectors, train_labels, test_labels = train_test_split(college_stats, successful, test_size = 0.25, train_size = 0.75)

In [11]:
logistic_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
logistic_result = logistic_model.fit()
print(logistic_result.summary())

Optimization terminated successfully.
         Current function value: 0.350464
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                Success   No. Observations:                   88
Model:                          Logit   Df Residuals:                       75
Method:                           MLE   Df Model:                           12
Date:                Sat, 15 Apr 2023   Pseudo R-squ.:                  0.4226
Time:                        22:41:34   Log-Likelihood:                -30.841
converged:                       True   LL-Null:                       -53.413
Covariance Type:            nonrobust   LLR p-value:                 9.736e-06
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const          138.0569    229.647      0.601      0.548    -312.043     588.157
Year            -0.1028